In [ ]:
import numpy as np

# Load the .npy file
# pointcloud_data = np.load('/Users/stevenymchang/Downloads/slam01_laneline_label.npy')
pointcloud_data = np.load(r"C:\Users\Bryan\Desktop\ece484\Final_Project\ece484_final\mapping_test\slam07_mapping_local.npy")

# Print the shape of the data (frames, channels, points, attributes)
print("Shape of the point cloud data:", pointcloud_data.shape)

Shape of the point cloud data: (632, 128, 1024, 7)


In [ ]:
from kiss_icp import Pipeline
import matplotlib.pyplot as plt

class FrameMapper:
    def __init__(self):
        self.pipeline = Pipeline()
        self.reference_frame = None
        self.global_map = None
        self.poses = []

    def process_data(self, data):
        """
        Process LiDAR data
        Input data shape: (632, 128, 1024, 7)
        """
        # Reshape all frames to (frames, N, 3) where N = 128*1024
        frames, channels, points, _ = data.shape
        reshaped_data = data[:, :, :, :3].reshape(frames, channels * points, 3)
        
        # Set first frame as reference
        self.reference_frame = reshaped_data[0]  # Shape: (N, 3)
        self.global_map = self.reference_frame
        self.poses.append(np.eye(4))
        
        # Process subsequent frames
        for frame_idx in range(1, frames):
            # Get current frame (already in shape (N, 3))
            current_frame = reshaped_data[frame_idx]
            
            # Get pose from KISS-ICP
            pose = self.pipeline.process_frame(current_frame)
            self.poses.append(pose)
            
            # Transform to reference frame
            global_points = self.transform_points(current_frame, pose)
            
            # Add to global map
            self.global_map = np.vstack([self.global_map, global_points])
            
            if frame_idx % 10 == 0:
                print(f"Processed frame {frame_idx}/{frames}")

    def transform_points(self, points, pose):
        """Transform points to global reference frame"""
        points_h = np.hstack([points, np.ones((len(points), 1))])
        global_points = (pose @ points_h.T).T
        return global_points[:, :3]
    
    def plot_global_map(self):
        """Plot all transformed frames in one visualization"""
        fig = plt.figure(figsize=(15, 5))
        
        # Top view (XY plane)
        plt.subplot(131)
        plt.scatter(self.global_map[:, 0], 
                   self.global_map[:, 1],
                   c='blue', s=1, alpha=0.1)  # Lower alpha for density visualization
        plt.xlabel('X (m)')
        plt.ylabel('Y (m)')
        plt.title('Top View - All Frames')
        plt.axis('equal')
        
        # Side view (XZ plane)
        plt.subplot(132)
        plt.scatter(self.global_map[:, 0], 
                   self.global_map[:, 2],
                   c='blue', s=1, alpha=0.1)
        plt.xlabel('X (m)')
        plt.ylabel('Z (m)')
        plt.title('Side View - All Frames')
        plt.axis('equal')
        
        # 3D view
        ax = plt.subplot(133, projection='3d')
        scatter = ax.scatter(self.global_map[:, 0], 
                           self.global_map[:, 1], 
                           self.global_map[:, 2],
                           c='blue', s=1, alpha=0.1)
        ax.set_xlabel('X (m)')
        ax.set_ylabel('Y (m)')
        ax.set_zlabel('Z (m)')
        plt.title('3D View - All Frames')
        
        plt.tight_layout()
        plt.show()

# Usage
if __name__ == "__main__":
    # Load data
    data = pointcloud_data
    
    # Create mapper and process
    mapper = FrameMapper()
    mapper.process_data(data)

    # Plot all frames mapped together
    mapper.plot_global_map()
    
    # # Save results
    # np.savez("mapping_results.npz",
    #          reference_frame=mapper.reference_frame,
    #          global_map=mapper.global_map,
    #          poses=mapper.poses)


In [6]:
# converting data
import numpy as np
import os

# Load your data (shape: 408, 128, 1024, 7)
data = pointcloud_data 

# Directory for output
output_dir = r"D:\484_final_project\dataset\sequences\06\velodyne"
os.makedirs(output_dir, exist_ok=True)
max = 65535.0

# Iterate over each frame and process
for frame_idx in range(data.shape[0]):
    frame_data = data[frame_idx]  # Shape (128, 1024, 7)
    
    # Extract x, y, z, and near IR (attributes 0, 1, 2, and 6)
    points = frame_data[:, :, [0, 1, 2, 6]]
    
    # Normalize near IR (attribute 6) to be between 0 and 1
    near_ir = points[:, :, 3]
    points[:, :, 3] = near_ir / max
    
    # Flatten channels and points to get shape (131072, 4)
    points = points.reshape(-1, 4)
    
    # Ensure the data is of type float32 for compatibility
    points = points.astype(np.float32)
    
    # Save each frame to a .bin file
    output_path = os.path.join(output_dir, f"{str(frame_idx).zfill(6)}.bin")
    points.tofile(output_path)

print("Conversion complete.")

Conversion complete.


In [1]:
# converting label
import numpy as np
import os

# Load your labels data (shape: 407, 128, 1024)
labels = np.load(r"D:\484_final_project\processed_point_cloud\slam06_predictions.npy")

# Directory for output
output_dir = r"D:\484_final_project\dataset\sequences\06"
os.makedirs(output_dir, exist_ok=True)

# Iterate over each frame and flatten the channels and points
for frame_idx in range(labels.shape[0]):
    frame_labels = labels[frame_idx]  # Shape (128, 1024)
    
    # Flatten the (128, 1024) to (131072,) for each frame
    flattened_labels = frame_labels.reshape(-1)  # Shape (131072,)
    
    # Ensure labels are stored as integers
    flattened_labels = flattened_labels.astype(np.int32)
    
    # Save the flattened labels as a .bin file
    labels_output_path = os.path.join(output_dir, f"{str(frame_idx).zfill(6)}.label")
    flattened_labels.tofile(labels_output_path)

print("Label conversion complete.")

Label conversion complete.
